# 테스트

평가 코드는 공정성을 위해 레퍼런스 코드 그대로 이용

In [1]:
import numpy as np
import os
from PIL import Image
import imageio
from glob import glob
from skimage.metrics import peak_signal_noise_ratio
from skimage.metrics import structural_similarity
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

In [2]:
def load_img(OUTPUT_DIR,expected_totalaug=0,test=False):

    split_list = []
    for i in (sorted(glob(OUTPUT_DIR + '/*'))):
        split_list.append(i)

    if test:
        pass
    else:
        if len(split_list) != expected_totalaug:
            print("Not fittable the number of augmented images !!")
            exit()
        else:
            print("Next Level : Split images Using Window, {} pic".format(len(split_list)))

    return split_list

def resize_image_by_pil(image, scale):
    width, height = image.shape[1], image.shape[0]
    new_width = int(width * scale)
    new_height = int(height * scale)
    method = Image.BICUBIC

    if len(image.shape) == 3 and image.shape[2] == 3:
        image = Image.fromarray(image, "RGB")
        image = image.resize([new_width, new_height], resample=method)
        image = np.asarray(image)
    elif len(image.shape) == 3 and image.shape[2] == 4:
        # the image may has an alpha channel
        image = Image.fromarray(image, "RGB")
        image = image.resize([new_width, new_height], resample=method)
        image = np.asarray(image)
    else:
        image = Image.fromarray(image.reshape(height, width))
        image = image.resize([new_width, new_height], resample=method)
        image = np.asarray(image)
        image = image.reshape(new_height, new_width, 1)
    return image


def save_image(filename, image, print_console=True):
    if len(image.shape) >= 3 and image.shape[0] == 1:
        image = image.reshape(image.shape[1], image.shape[2])

    directory = os.path.dirname(filename)
    if directory != "" and not os.path.exists(directory):
        os.makedirs(directory)

    image = image.astype(np.uint8)
    if len(image.shape) >= 3 and image.shape[0] == 3:
        image = Image.fromarray(image, mode="RGB")
    else:
        image = Image.fromarray(image)
    imageio.imwrite(filename, image)

    if print_console:
        print("Saved [%s]" % filename)


def set_image_alignment(image, alignment):
    alignment = int(alignment)
    width, height = image.shape[1], image.shape[0]
    width = (width // alignment) * alignment
    height = (height // alignment) * alignment

    if image.shape[1] != width or image.shape[0] != height:
        image = image[:height, :width, :]

    if len(image.shape) >= 3 and image.shape[2] >= 4:
        image = image[:, :, 0:3]

    return image

def convert_rgb_to_y(image, jpeg_mode=False, max_value=255.0):
    if len(image.shape) <= 2 or image.shape[2] == 1:
        return image

    if jpeg_mode:
        xform = np.array([[0.299, 0.587, 0.114]])
        y_image = image.dot(xform.T)
    else:
        xform = np.array([[65.481 / 256.0, 128.553 / 256.0, 24.966 / 256.0]])
        y_image = image.dot(xform.T) + (16.0 * max_value / 256.0)

    return y_image

def convert_rgb_to_ycbcr(image):
    if len(image.shape) < 2 or image.shape[2] == 1:
        return image

    xform = np.array(
        [[65.738 / 256.0, 129.057 / 256.0, 25.064 / 256.0],
         [- 37.945 / 256.0, - 74.494 / 256.0, 112.439 / 256.0],
         [112.439 / 256.0, - 94.154 / 256.0, - 18.285 / 256.0]])

    ycbcr_image = image.dot(xform.T)
    ycbcr_image[:, :, 0] += 16.0
    ycbcr_image[:, :, [1, 2]] += 128.0

    return ycbcr_image


def convert_y_and_cbcr_to_rgb(y_image, cbcr_image):
    print(cbcr_image.shape)
    if len(y_image.shape) <= 2:
        y_image = y_image.reshape[y_image.shape[0], y_image.shape[1], 1]

    if len(y_image.shape) == 3 and y_image.shape[2] == 3:
        y_image = y_image[:, :, 0:1]

    ycbcr_image = np.zeros([y_image.shape[0], y_image.shape[1], 3])
    ycbcr_image[:, :, 0] = y_image[:, :, 0]
    ycbcr_image[:, :, 1:3] = cbcr_image[:, :, 0:2]

    return convert_ycbcr_to_rgb(ycbcr_image)

def convert_ycbcr_to_rgb(ycbcr_image):
    rgb_image = np.zeros([ycbcr_image.shape[0], ycbcr_image.shape[1], 3])  # type: np.ndarray

    rgb_image[:, :, 0] = ycbcr_image[:, :, 0] - 16.0
    rgb_image[:, :, [1, 2]] = ycbcr_image[:, :, [1, 2]] - 128.0
    xform = np.array(
        [[298.082 / 256.0, 0, 408.583 / 256.0],
         [298.082 / 256.0, -100.291 / 256.0, -208.120 / 256.0],
         [298.082 / 256.0, 516.412 / 256.0, 0]])
    rgb_image = rgb_image.dot(xform.T)

    return rgb_image
def get_split_images(image, window_size, stride=None, enable_duplicate=True):
    if len(image.shape) == 3 and image.shape[2] == 1:
        image = image.reshape(image.shape[0], image.shape[1])
    #print(image.shape)
    window_size = int(window_size)
    size = image.itemsize
    height, width = image.shape
    if stride is None:
        stride = window_size
    else:
        stride = int(stride)

    if height < window_size or width < window_size:
        return None

    new_height = 1 + (height - window_size) // stride
    new_width = 1 + (width - window_size) //  stride

    shape = (new_height, new_width, window_size, window_size)
    strides = size * np.array([width * stride, stride, width, 1])
    windows = np.lib.stride_tricks.as_strided(image, shape=shape, strides=strides)


    windows = windows.reshape(windows.shape[0] * windows.shape[1],1, windows.shape[2], windows.shape[3])

    if enable_duplicate:
        extra_windows = []
        if (height - window_size) % stride != 0:
            for x in range(0, width - window_size, stride):
                extra_windows.append(image[height - window_size - 1:height - 1, x:x + window_size:])

        if (width - window_size) % stride != 0:
            for y in range(0, height - window_size, stride):
                extra_windows.append(image[y: y + window_size, width - window_size - 1:width - 1])

        if len(extra_windows) > 0:
            org_size = windows.shape[0]
            windows = np.resize(windows,
                                [org_size + len(extra_windows), windows.shape[1], windows.shape[2], windows.shape[3]])
            for i in range(len(extra_windows)):
                extra_windows[i] = extra_windows[i].reshape([1,extra_windows[i].shape[0], extra_windows[i].shape[1]])
                windows[org_size + i] = extra_windows[i]

    return windows

def GPU_AVAILABLE():
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    print('Device:', device)  # 출력결과: cuda
    print('Count of using GPUs:', torch.cuda.device_count())  # 출력결과: 2 (2, 3 두개 사용하므로)
    print('Current cuda device:', torch.cuda.current_device())  # 출력결과: 2 (2, 3 중 앞의 GPU #2 의미)
    return device

def compute_psnr_and_ssim(image1, image2, border_size=0):
    """
    Computes PSNR and SSIM index from 2 images.
    We round it and clip to 0 - 255. Then shave 'scale' pixels from each border.
    """
    if len(image1.shape) == 2:
        image1 = image1.reshape(image1.shape[0], image1.shape[1], 1)
    if len(image2.shape) == 2:
        image2 = image2.reshape(image2.shape[0], image2.shape[1], 1)

    if image1.shape[0] != image2.shape[0] or image1.shape[1] != image2.shape[1] or image1.shape[2] != image2.shape[2]:
        return None

    image1 = trim_image_as_file(image1)
    image2 = trim_image_as_file(image2)

    if border_size > 0:
        image1 = image1[border_size:-border_size, border_size:-border_size, :]
        image2 = image2[border_size:-border_size, border_size:-border_size, :]

    psnr = peak_signal_noise_ratio(image1, image2, data_range=255)
    ssim = structural_similarity(image1, image2, win_size=11, gaussian_weights=True, multichannel=True, K1=0.01, K2=0.03,
                        sigma=1.5, data_range=255)
    return psnr, ssim


def trim_image_as_file(image):
    image = np.rint(image)
    image = np.clip(image, 0, 255)
    if image.dtype != np.float32:
        image = image.astype(np.float32)
    return image



In [3]:
class DCSCN(nn.Module):
    
    def __init__(self, kernel_size = 3, n_channels = 64):
        super(DCSCN, self).__init__()
                
        # common layer
        self.drop = nn.Dropout(p=0.8)
        self.prelu = nn.PReLU() # nn.PReLU(self.conv1(x)) 방식 이용 불가. 별도 선언.
        
        # feature extraction layer
        self.conv1 = nn.Conv2d(1, 196, kernel_size, stride = 1, padding = 1, padding_mode = "replicate", bias = True)
        self.conv2 = nn.Conv2d(196, 166, kernel_size, stride = 1, padding = 1, padding_mode = "replicate", bias = True)
        self.conv3 = nn.Conv2d(166, 148, kernel_size, stride = 1, padding = 1, padding_mode = "replicate", bias = True)
        self.conv4 = nn.Conv2d(148, 133, kernel_size, stride = 1, padding = 1, padding_mode = "replicate", bias = True)
        self.conv5 = nn.Conv2d(133, 120, kernel_size, stride = 1, padding = 1, padding_mode = "replicate", bias = True)
        self.conv6 = nn.Conv2d(120, 108, kernel_size, stride = 1, padding = 1, padding_mode = "replicate", bias = True)
        self.conv7 = nn.Conv2d(108, 97, kernel_size, stride = 1, padding = 1, padding_mode = "replicate", bias = True)
        self.conv8 = nn.Conv2d(97, 86, kernel_size, stride = 1, padding = 1, padding_mode = "replicate", bias = True)
        self.conv9 = nn.Conv2d(86, 76, kernel_size, stride = 1, padding = 1, padding_mode = "replicate", bias = True)
        self.conv10 = nn.Conv2d(76, 66, kernel_size, stride = 1, padding = 1, padding_mode = "replicate", bias = True)
        self.conv11 = nn.Conv2d(66, 57, kernel_size, stride = 1, padding = 1, padding_mode = "replicate", bias = True)
        self.conv12 = nn.Conv2d(57, 48, kernel_size, stride = 1, padding = 1, padding_mode = "replicate", bias = True)
        
        ## he initialization
        nn.init.kaiming_normal_(self.conv1.weight)
        nn.init.kaiming_normal_(self.conv2.weight)
        nn.init.kaiming_normal_(self.conv3.weight)
        nn.init.kaiming_normal_(self.conv4.weight)
        nn.init.kaiming_normal_(self.conv5.weight)
        nn.init.kaiming_normal_(self.conv6.weight)
        nn.init.kaiming_normal_(self.conv7.weight)
        nn.init.kaiming_normal_(self.conv8.weight)
        nn.init.kaiming_normal_(self.conv9.weight)
        nn.init.kaiming_normal_(self.conv10.weight)
        nn.init.kaiming_normal_(self.conv11.weight)
        nn.init.kaiming_normal_(self.conv12.weight)
        
        # reconstruction layer
        self.A1 = nn.Conv2d(1301, 64, 1, stride = 1, bias = True)
        self.B1 = nn.Conv2d(1301, 32, 1, stride = 1, bias = True)
        self.B2 = nn.Conv2d(32, 32, kernel_size, stride = 1, padding=1,padding_mode="replicate",bias=True)
        
        ## he initialization
        nn.init.kaiming_normal_(self.A1.weight)
        nn.init.kaiming_normal_(self.B1.weight)
        nn.init.kaiming_normal_(self.B2.weight)
        
        # feature extraction layer
        self.UPsample = nn.Conv2d(96, 2*2*96, kernel_size, stride = 1, padding =1, padding_mode = "replicate", bias = True)
        self.pixelshuffle = nn.PixelShuffle(2)
        
        self.R = nn.Conv2d(96, 1, kernel_size, stride = 1, padding = 1, padding_mode = "replicate", bias = True)
        
        ## he initialization
        nn.init.kaiming_normal_(self.UPsample.weight)
        nn.init.kaiming_normal_(self.R.weight)
        
    def forward(self, x):
                
        # feature extraction network
        skip1 = self.drop(self.prelu(self.conv1(x)))
        skip2 = self.drop(self.prelu(self.conv2(skip1)))
        skip3 = self.drop(self.prelu(self.conv3(skip2)))
        skip4 = self.drop(self.prelu(self.conv4(skip3)))
        skip5 = self.drop(self.prelu(self.conv5(skip4)))
        skip6 = self.drop(self.prelu(self.conv6(skip5)))
        skip7 = self.drop(self.prelu(self.conv7(skip6)))
        skip8 = self.drop(self.prelu(self.conv8(skip7)))
        skip9 = self.drop(self.prelu(self.conv9(skip8)))
        skip10 = self.drop(self.prelu(self.conv10(skip9)))
        skip11 = self.drop(self.prelu(self.conv11(skip10)))
        skip12 = self.drop(self.prelu(self.conv12(skip11)))
        
        # reconstruction network
        recon_input = torch.cat([skip1, skip2, skip3, skip4, skip5, skip6, skip7, skip8, skip9, skip10, skip11, skip12], dim = 1)
        
        A1_out = self.drop(self.prelu(self.A1(recon_input)))
        
        B1_out = self.drop(self.prelu(self.B1(recon_input)))
        B2_out = self.drop(self.prelu(self.B2(B1_out)))
        
        recon_output = torch.cat([A1_out, B2_out], dim = 1)
        
        # up-sampling network
        UPsample_out = self.drop(self.pixelshuffle(self.UPsample(recon_output)))
        R_out = self.R(UPsample_out)
        
        return R_out

In [4]:
# Checking GPU Available

# splited 된 그림을 보길 원하시면 batch_picture_save_flag 를 1 로 바꾸시면 됩니다.
# 경로 : augmented_data/train_sr
batch_picture_save_flag = 0
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)  # 출력결과: cuda
print('Count of using GPUs:', torch.cuda.device_count())  # 출력결과: 2 (2, 3 두개 사용하므로)
print('Current cuda device:', torch.cuda.current_device())  # 출력결과: 2 (2, 3 중 앞의 GPU #2 의미)

# Configure Data Augmentation

DATA_DIR = ['data/bsd200', 'data/yang91']
OUTPUT_DIR = 'augmented_data/train_org/'

# Split Parameters

BICUBIC_DIR = 'augmented_data/train_sr/LRBICUBIC'
LRX2_DIR = 'augmented_data/train_sr/LRX2'
HR_DIR = 'augmented_data/train_sr/HR'

lr_batch_size = 32
scale = 2

Device: cuda
Count of using GPUs: 1
Current cuda device: 0


In [5]:
def test(model_path,lr,bi):

    device = GPU_AVAILABLE()

    if len(lr.shape) == 3 and lr.shape[2] == 1:
        lr_luma = lr.reshape(1,1,lr.shape[0], lr.shape[1])
    if len(bi.shape) == 3 and bi.shape[2] == 1:
        bi_luma = bi.reshape(1,1,bi.shape[0], bi.shape[1])

    lr_luma = torch.FloatTensor(lr_luma).to(device)
    bi_luma = torch.FloatTensor(bi_luma).to(device)
    
    load_model = torch.load(model_path).to(device)
    load_model.eval()

    with torch.no_grad():
        pred = load_model(lr_luma)
        pred += bi_luma

    pred = pred.detach().cpu().numpy()
    pred = pred.reshape(pred.shape[2],pred.shape[3],1)
    return pred

In [6]:
# model load

load_path = 'save_model/DCSCN_V2_e100_lr0.0001.pt'

# test image load

test_dir = 'data/set5'
test_list = load_img(test_dir,test=True)
scale_factor = 2
output_dir = 'test/'

AVG_PSNR = []
AVG_SSIM = []
for i in test_list:
    file_name = os.path.basename(i)
    file_name,ext = os.path.splitext(file_name)

    img = imageio.imread(i)
    lr_img = resize_image_by_pil(img,1/scale_factor)
    bi_img = resize_image_by_pil(lr_img,scale_factor)

    y_img = convert_rgb_to_y(img)
    y_lr_img = resize_image_by_pil(y_img,1/scale_factor)
    y_bi_img = resize_image_by_pil(y_lr_img,scale_factor)
    ycbcr_bi_img = convert_rgb_to_ycbcr(bi_img)

    recon = test(load_path,y_lr_img,y_bi_img)
    recon_rgb = convert_y_and_cbcr_to_rgb(recon,ycbcr_bi_img[:,:,1:3])

    bicubic_rgb = convert_y_and_cbcr_to_rgb(y_bi_img,ycbcr_bi_img[:,:,1:3])

    luma_psnr,luma_ssim = compute_psnr_and_ssim(y_img, recon, 2+scale_factor)
    luma_bipsnr, luma_bissim = compute_psnr_and_ssim(y_img, y_bi_img, 0)

    print("luma_recon : {} / {} luma_bicubic : {} / {}".format(luma_psnr, luma_ssim, luma_bipsnr, luma_bissim))

    org_name = output_dir+file_name + '_org' + ext
    recon_save_name = output_dir+file_name + '_recon' + ext
    bi_save_name = output_dir+file_name + '_bi' + ext

    recon_rgb = np.clip(recon_rgb,0,255)
    bicubic_rgb = np.clip(bicubic_rgb,0,255)

    AVG_PSNR.append(luma_psnr)
    AVG_SSIM.append(luma_ssim)

avg_psnr = sum(AVG_PSNR)/len(AVG_PSNR)
avg_ssim = sum(AVG_SSIM)/len(AVG_SSIM)

print("avg_psnr / avg_ssim : {} / {}".format(avg_psnr,avg_ssim))

Device: cuda
Count of using GPUs: 1
Current cuda device: 0
(512, 512, 2)
(512, 512, 2)
luma_recon : 38.51050064774633 / 0.9638012975184947 luma_bicubic : 37.12439095115347 / 0.9520453248230841
Device: cuda
Count of using GPUs: 1
Current cuda device: 0
(288, 288, 2)
(288, 288, 2)
luma_recon : 40.95891910657626 / 0.9859568622289656 luma_bicubic : 36.73963134778147 / 0.9719822655200628
Device: cuda
Count of using GPUs: 1
Current cuda device: 0
(256, 256, 2)
(256, 256, 2)
luma_recon : 31.56711515586748 / 0.9608356546436907 luma_bicubic : 27.485831267157707 / 0.9150048104031713
Device: cuda
Count of using GPUs: 1
Current cuda device: 0
(280, 280, 2)
(280, 280, 2)
luma_recon : 35.77474722001781 / 0.8840012023219526 luma_bicubic : 34.90877421068931 / 0.8621171629666867
Device: cuda
Count of using GPUs: 1
Current cuda device: 0
(344, 228, 2)
(344, 228, 2)
luma_recon : 35.251708314441935 / 0.9677848159034801 luma_bicubic : 32.1838099118808 / 0.9474995647197535
avg_psnr / avg_ssim : 36.412598088